# Advent of Code
## Day 8: 
### Part One

Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (`acc`, `jmp`, or `nop`) and an argument (a signed number like `+4` or `-20`).

`acc` increases or decreases a single global value called the accumulator by the value given in the argument. For example, `acc +7` would increase the accumulator by 7. The accumulator starts at `0`. After an `acc` instruction, the instruction immediately below it is executed next.
`jmp` jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, `jmp +2` would skip the next instruction, `jmp +1` would continue to the instruction immediately below it, and `jmp -20` would cause the instruction 20 lines above to be executed next.
`nop` stands for No OPeration - it does nothing. The instruction immediately below it is executed next.
For example, consider the following program:

>```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
>```

These instructions are visited in this order:

>```
nop +0  | 1
acc +1  | 2, 8(!)
jmp +4  | 3
acc +3  | 6
jmp -3  | 7
acc -99 |
acc +1  | 4
jmp -4  | 5
acc +6  |
>```

First, the `nop +0` does nothing. Then, the accumulator is increased from 0 to 1 (`acc +1`) and `jmp +4` sets the next instruction to the other `acc +1` near the bottom. After it increases the accumulator from 1 to 2, `jmp -4` executes, setting the next instruction to the only `acc +3`. It sets the accumulator to 5, and `jmp -3` causes the program to continue back at the first `acc +1`.

This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, you know it will never terminate.

Immediately before the program would run an instruction a second time, the value in the accumulator is `5`.

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

---

In [1]:
# Read in the inputs as an (operation, value) pair
inputs = [tuple(i[:-1].split(' ')) for i in open("Day08_input.txt").readlines()]

# Clean the inputs so that each value is an integer
for i, (instruction, val) in enumerate(inputs):
    inputs[i] = (instruction, int(val))

In [2]:
# Initialise variables
lines_executed = set()  # this will store all the executed lines
acc = 0                 # acc is initialised to 0
line_counter = 0        # start by executing line 0

# While we haven't repeated any lines and the line counter is less than the number of inputs
while (line_counter not in lines_executed) and (line_counter < len(inputs)):
    # Add the executing line to the set of executed lines
    lines_executed.add(line_counter)
    # Read the instruction on this line
    instruction, val = inputs[line_counter]
    
    # Parse the instruction and execute it as per described
    if instruction == 'acc':
        acc += val
        line_counter += 1
    elif instruction == 'jmp':
        line_counter += val
    elif instruction == 'nop':
        line_counter += 1
    
# Once we repeat a line, this will break and give us the first answer
print(f"Program will repeat on line {line_counter:,} with accumulator value {acc:,}")

Program will repeat on line 178 with accumulator value 1,614


---

### Part Two

After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a `jmp` is supposed to be a `nop`, or a `nop` is supposed to be a `jmp`. (No `acc` instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one `jmp` or `nop`, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:

>```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
>```

If you change the first instruction from `nop +0` to `jmp +0`, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the jmp instructions, the program will still eventually find another `jmp` instruction and loop forever.

However, if you change the second-to-last instruction (from `jmp -4` to `nop -4`), the program terminates! The instructions are visited in this order:

>```
nop +0  | 1
acc +1  | 2
jmp +4  | 3
acc +3  |
jmp -3  |
acc -99 |
acc +1  | 4
nop -4  | 5
acc +6  | 6
>```

After the last instruction (`acc +6`), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value `8` (`acc +1`, `acc +1`, `acc +6`).

Fix the program so that it terminates normally by changing exactly one `jmp` (to `nop`) or `nop` (to `jmp`). What is the value of the accumulator after the program terminates?

---

In [3]:
# This custom exception will help us diagnose the issue
class InfiniteLoop(Exception):
    pass

# This function will take a list of instructions and run it until it hits an infinite loop
def test_run(script):
    """
    Takes a list of instructions and either returns the accumulator after the script completes
    or raises an InfiniteLoop error.
    
    :param script: list, an ordered list of instructions to execute
    :return: int, the accumulator value after a successful execution
    :raises: InfiniteLoop, an error if the test is unsuccessful
    """
    # Initialise variables
    lines_executed = set()  # this will store all the executed lines
    acc = 0                 # acc is initialised to 0
    line_counter = 0        # start by executing line 0
    
    # While we haven't repeated any lines and the line counter is less than the number of inputs
    while (line_counter not in lines_executed) and (line_counter < len(script)):
        # Add the executing line to the set of executed lines
        lines_executed.add(line_counter)
        # Read the instruction on this line
        instruction, val = script[line_counter]

        # Parse the instruction and execute it as per described
        if instruction == 'acc':
            acc += val
            line_counter += 1
        elif instruction == 'jmp':
            line_counter += val
        elif instruction == 'nop':
            line_counter += 1
    
    # if the script has successfully completed
    if line_counter >= len(script):
        # return accumulator values
        return acc
    else:
        # otherwise, raise an InfiniteLoop error
        raise InfiniteLoop("This script contains an infinite loop")

In [4]:
# Since we didn't do it in part one, let's now REMEMBER the order in which operations are executed
# Then we only need to check those lines to see if they change
ordered_operations = []
line_counter = 0
acc = 0

# Then run the script as before...
while line_counter not in ordered_operations:
    # Note: this time we're appending them in order
    ordered_operations.append(line_counter)
    instruction, val = inputs[line_counter]
    
    if instruction == 'acc':
        acc += val
        line_counter += 1
    elif instruction == 'jmp':
        line_counter += val
    elif instruction == 'nop':
        line_counter += 1

In [5]:
# Now let's debug.
# I want to know the accumulator value, the line number that's changed,
# the original instruction and the instruction it was modified to...
# So I'll set them to be None for now.
acc = None
changed_line = None
original_input = None
modified_input = None

# Let's reverse the order we check for corrupt operations in since I get the
# (completely unfounded) hunch it will be close to the end.
for i in reversed(ordered_operations):
    # copy the inputs so that we don't corrupt the original instructions
    modified_inputs = inputs.copy()
    # and read in a candidate instruction to change
    instruction, val = inputs[i]
    
    # If it's a jump instruction
    if instruction == 'jmp':
        # Change it to a no operation instruction
        modified_inputs[i] = ('nop', val)
        # And test if the code now runs
        try:
            # If it does run, record the acc value
            acc = test_run(modified_inputs)
        except InfiniteLoop:
            # Else, keep looking
            pass
    # If it's a no operation instruction
    elif instruction == 'nop':
        # Change it to a jump instruction
        modified_inputs[i] = ('jmp', val)
        # And test if the code now runs
        try:
            # If it does run, record the acc value
            acc = test_run(modified_inputs)
        except InfiniteLoop:
            # Else, keep looking
            pass
    # If it's an accumulate instruction
    else:
        # We don't need to worry about it
        pass
    
    # If we've modified a command and the script now runs
    if acc is not None:
        # record the variables
        changed_line = i
        original_input = (instruction,val)
        modified_input = modified_inputs[changed_line]
        # and exit the loop
        break
    
# Finally, print the solution to the screen.
print(
    f"After changing line {changed_line} from {original_input} to {modified_input}"
    f" the program terminates with accumulator value {acc:,}."
)

After changing line 247 from ('jmp', 209) to ('nop', 209) the program terminates with accumulator value 1,260.


---